In [ ]:
import datasets
from datasets import Split

# Load Geo-Bench
train_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'train')
test_dataset = datasets.load_dataset("GEO-optim/geo-bench", 'test')

In [ ]:
print(train_dataset['train'][0])
print(test_dataset['test'][0]['sources'][0])
print(test_dataset['test'][0])

In [11]:
for i, k in enumerate(test_dataset['test']):    
        print(i, k)
        break

NameError: name 'test_dataset' is not defined

In [ ]:
from google import genai
from google.genai import types
client = genai.Client()
response = client.models.generate_content(
    model="gemini-2.5-pro", 
    contents="Explain how AI works in short",
)
print(response.text)

In [28]:
import google.generativeai as googlegenai
model = 'gemini-2.5-pro'
model_client = googlegenai.GenerativeModel(model)
response = model_client.generate_content(
                    "Explain how AI works in short",
                    generation_config={
                        'temperature': 0.5,
                        'top_p': 1.0,
                        'max_output_tokens': 1024,
                    }
                )
response

KeyboardInterrupt: 